In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
data=pd.read_csv('Database.csv',index_col=None)

In [5]:
Label=data.iloc[:,-22:-3]
for i in range(Label.shape[0]):
    for j in range(Label.shape[1]):
        if Label.iloc[i,j]!=0:
            Label.iloc[i,j]=1

In [6]:
x=data.iloc[:,3:13]
x_encode=pd.get_dummies(x,columns=x.columns)
x_encode.shape

(99, 82)

In [7]:
from sklearn.decomposition import PCA
pca=PCA(n_components=0.3)
x_feature=pca.fit_transform(x_encode.values)

In [8]:
X_train,X_test,Y_train,Y_test=train_test_split(x_feature,Label,test_size=0.25,shuffle=True)

In [9]:
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
##from xgboost import XGBClassifier
##from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
Multi_Classifier=MultiOutputClassifier(RandomForestClassifier())
clf_classification=Pipeline([('Classifier',Multi_Classifier)])
clf_classification.fit(X_train,Y_train)
y_pred=pd.DataFrame(Multi_Classifier.predict(X_test))
y_pred.columns=pd.DataFrame(Label).columns
y_pred.to_csv('D:/OneDrive/SRTP/output_pred.csv',encoding='utf-8_sig')
Y_test.to_csv('D:/OneDrive/SRTP/output_test.csv',encoding='utf-8_sig')

In [10]:
from sklearn.metrics import classification_report
print(classification_report(Y_test,y_pred))
from sklearn.metrics import hamming_loss
print(hamming_loss(Y_test,y_pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.46      0.55      0.50        11
           2       0.25      0.11      0.15         9
           3       0.43      0.50      0.46         6
           4       0.33      0.50      0.40         2
           5       1.00      0.25      0.40         4
           6       0.84      1.00      0.91        21
           7       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         3
           9       0.71      0.38      0.50        13
          10       0.60      0.67      0.63         9
          11       0.00      0.00      0.00         1
          12       0.82      0.95      0.88        19
          13       0.00      0.00      0.00         0
          14       0.20      0.25      0.22         4
          15       0.95      1.00      0.98        20
          16       0.00      0.00      0.00         0
          17       0.00    